In [ ]:
import lux
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/lux-org/lux-datasets/master/data/college.csv")

In [ ]:
df

## Trying out tracing

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/lux-org/lux-datasets/master/data/car.csv")
df["Year"] = pd.to_datetime(df["Year"], format='%Y')

In [ ]:
import sys

In [ ]:
import pickle as pkl
relevant_lines=[]
def profile_func( frame, event, arg):
    #Profile functions should have three arguments: frame, event, and arg. 
    #frame is the current stack frame. 
    #event is a string: 'call', 'return', 'c_call', 'c_return', or 'c_exception'.
    #arg depends on the event type.
    try: 
        if event == 'line' :
            fcode = frame.f_code             
            line_no = frame.f_lineno
            func_name = fcode.co_name
            #includeMod = ['lux/vis', 'lux/action', 'lux/vislib', 'lux/executor', 'lux/interestingness']
            includeMod = ['lux/vislib','lux/executor']
            if any(x in frame.f_code.co_filename for x in includeMod):
                if (func_name!="<lambda>"): #ignore one-liner lambda functions (repeated line events)
                    relevant_lines.append([frame.f_code.co_filename,func_name,line_no])
#                     print (f"{frame.f_code.co_filename}--{func_name}--{line_no}")  
#                 print (fcode)
                
                #print (inspect.getsourcelines(fcode))
#                 pkl.dump(frame.f_code,open("frame.pkl",'wb'))
    except:
        # pass  # just swallow errors to avoid interference with traced processes
        raise  # for debugging
    return profile_func

In [ ]:
def start_tracing():
    print ("-----------start_tracing-----------")
    # Implement python source debugger: https://docs.python.org/3/library/sys.html#sys.settrace
    # setprofile faster than settrace (only go through I/O)
    sys.settrace(profile_func) 
def stop_tracing():
    print ("-----------stop_tracing-----------")
    sys.settrace(None) 

In [ ]:
start_tracing()

In [ ]:
%time 
from lux.vis.Vis import Vis
vis  = Vis(["Horsepower"],df)
vis

In [ ]:
stop_tracing()

In [ ]:
%time
from lux.vis.Vis import Vis
vis  = Vis(["Horsepower"],df)
vis

In [ ]:
print (vis.to_Altair())

In [ ]:
selected = ""
for line in relevant_lines:
    filename = line[0]
    funcname = line[1]
    line_no = line[2]-1
    if "Executor.py" not in filename or ("Executor.py" in filename and "execute" in funcname):
        codelines = open(filename).readlines()# TODO: do sharing of file content here
        if (funcname not in ['__init__']):
            code = codelines[line_no]
            ignore_construct = ['if','elif','for','return'] # prune out these control flow programming constructs                    
            ignore_lux_keyword = ['self.code','self.name','__init__','PandasExecutor.',"'''",'self.output_type']# Lux-specific keywords to ignore
            ignore = ignore_construct+ignore_lux_keyword
            if not any(construct in code for construct in ignore):
                print (f"{filename}--{funcname}--{line_no}")  
                print (codelines[line_no].lstrip())
                selected+=codelines[line_no].lstrip()

In [ ]:
selected = selected.replace("vis._vis_data","visdf").replace('vis.data','visdf').replace("self.data","visdf").replace('self.chart',"chart").replace('self.vis','vis')

In [ ]:
print (selected)

In [ ]:
from lux.utils import utils
from lux.vislib.altair.Histogram import Histogram

SAMPLE_START = 10000
SAMPLE_CAP = 30000
SAMPLE_FRAC = 0.75
df._sampled = df
visdf = df._sampled
visdf is not None
filters = utils.get_filter_specs(vis._inferred_intent)
attributes = set([])
attr = "Horsepower"
attributes.add(attr)
visdf = visdf[list(attributes)]
import numpy as np
bin_attribute = list(filter(lambda x: x.bin_size != 0, vis._inferred_intent))[0]
bin_attr = bin_attribute.attribute
series = visdf[bin_attr].dropna()
counts, bin_edges = np.histogram(series, bins=bin_attribute.bin_size)
bin_center = np.mean(np.vstack([bin_edges[0:-1], bin_edges[1:]]), axis=0)
binned_result = np.array([bin_center, counts]).T
visdf = pd.DataFrame(binned_result, columns=[bin_attr, "Number of Records"])
visdf[attr].iloc[0], pd.Period
visdf[attr].iloc[0], pd.Interval
visdf[attr].iloc[0], pd.Period
visdf[attr].iloc[0], pd.Interval
chart = Histogram(vis)
# self.tooltip = False
measure = vis.get_attr_by_data_model("measure", exclude_record=True)[0]
msr_attr = vis.get_attr_by_channel(measure.channel)[0]
x_min = vis.min_max[msr_attr.attribute][0]
x_max = vis.min_max[msr_attr.attribute][1]
x_range = abs(max(visdf[msr_attr.attribute]) - min(visdf[msr_attr.attribute]))
plot_range = abs(x_max - x_min)
markbar = x_range / plot_range * 12
# alt.Chart(visdf).mark_bar(size=markbar)
# alt.X(
# msr_attr.attribute,
# title=f"{msr_attr.attribute} (binned)",
# bin=alt.Bin(binned=True),
# type=msr_attr.data_type,
# axis=alt.Axis(labelOverlap=True),
# scale=alt.Scale(domain=(x_min, x_max)),
# alt.Y("Number of Records", type="quantitative")
# """
# color_attr = vis.get_attr_by_channel("color")
# chart_title = vis.title
# chart = chart.configure_title(fontWeight=500, fontSize=13, font="Helvetica Neue")
# chart = chart.configure_axis(
# titleFontWeight=500,
# titleFontSize=11,
# titleFont="Helvetica Neue",
# labelFontWeight=400,
# labelFontSize=9,
# labelFont="Helvetica Neue",
# labelColor="#505050",
# chart = chart.configure_legend(
# titleFontWeight=500,
# titleFontSize=10,
# titleFont="Helvetica Neue",
# labelFontWeight=400,
# labelFontSize=9,
# labelFont="Helvetica Neue",
# chart = chart.properties(width=160, height=150)
# "\nchart = chart.configure_title(fontWeight=500,fontSize=13,font='Helvetica Neue')\n"
# chart_dict = chart.chart.to_dict()
chart.vis